In [1]:
import os
import csv
import requests
import base64
import re
import Levenshtein

In [2]:
# === KONFIGURASI ===
image_dir = "test"
label_csv = "label.csv"
output_csv = "results.csv"
lmstudio_url = "http://localhost:1234/v1/chat/completions"
model_name = "llava-llama-3-8b-v1_1"
# model_name = "bakllava1-1.5-mistral-7b"


In [3]:
# === LOAD LABEL MANUAL ===
def load_labels(label_file):
    label_map = {}
    with open(label_file, newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            fname = row["image"].strip()
            gt = row["ground_truth"].strip().upper()
            label_map[fname] = re.sub(r'[^A-Z0-9]', '', gt)  # Bersihkan GT
    return label_map


In [4]:
# === ENCODE IMAGE BASE64 ===
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')


In [5]:
# === HITUNG CER ===
def cer_score(gt, pred):
    if len(gt) == 0:
        return 1.0
    distance = Levenshtein.distance(gt, pred)
    return round(distance / len(gt), 3)


In [6]:
# === INFER KE LM STUDIO ===
def ocr_infer(base64_image):
    headers = {"Content-Type": "application/json"}
    prompt = (
    "You are a specialized OCR model for Indonesian vehicle license plates.\n"
    "ONLY return the license plate number as a single string, with no extra explanation or punctuation.\n"
    "The format may include letters and numbers (e.g., B1234XYZ, D 123 AB, AB1234CD).\n"
    "Respond with only the plate number. Do NOT include any commentary, description, or full sentences."

    )
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                    },
                    {
                        "type": "text",
                        "text": "What is the license plate number in this image? Only respond with the number."
                    }
                ]
            }
        ],
        "temperature": 0,
        "max_tokens": 30
    }
    try:
        response = requests.post(lmstudio_url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"ERROR: {e}"

In [7]:
# === FILTER DENGAN REGEX ===
def extract_plate(text):
    text = text.upper()
    match = re.search(r"[A-Z]{1,2}[0-9]{1,4}[A-Z]{0,3}", text)
    return match.group(0) if match else re.sub(r'[^A-Z0-9]', '', text)

In [8]:
# === MAIN ===
def main():
    label_map = load_labels(label_csv)
    total_cer = 0
    total_images = 0

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["image", "ground_truth", "prediction", "CER_score"])

        for filename in sorted(os.listdir(image_dir)):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(image_dir, filename)
                gt = label_map.get(filename, "").upper()

                if not gt:
                    print(f"⚠️ Ground truth tidak ditemukan: {filename}")
                    continue

                try:
                    b64_img = encode_image(img_path)
                    raw_pred = ocr_infer(b64_img)

                    if raw_pred.startswith("ERROR"):
                        cleaned_pred = "ERROR"
                        cer = 1.0
                    else:
                        cleaned_pred = extract_plate(raw_pred)
                        cer = cer_score(gt, cleaned_pred)

                    total_cer += cer
                    total_images += 1

                    print(f"🔍 {filename}")
                    print(f"   ✅ Prediksi: {cleaned_pred} | GT: {gt} | CER: {cer}\n")
                    writer.writerow([filename, gt, cleaned_pred, cer])

                except Exception as e:
                    print(f"❌ {filename} → UNHANDLED ERROR: {e}")
                    writer.writerow([filename, gt, "UNHANDLED ERROR", 1.0])
                    total_cer += 1.0
                    total_images += 1

    avg_cer = round(total_cer / total_images, 3) if total_images else 1.0
    print(f"\n📊 Rata-rata CER dari {total_images} gambar: {avg_cer}")

if __name__ == "__main__":
    main()


🔍 test001_1.jpg
   ✅ Prediksi: 5140BC | GT: B9140BCD | CER: 0.375

🔍 test001_2.jpg
   ✅ Prediksi: B2407UZO | GT: B2407UZO | CER: 0.0

🔍 test001_3.jpg
   ✅ Prediksi: B2842PKM | GT: B2642PKM | CER: 0.125

🔍 test002_1.jpg
   ✅ Prediksi: BC1352AE | GT: BG1352AE | CER: 0.125

🔍 test003_1.jpg
   ✅ Prediksi: B2663UZF | GT: B2634UZF | CER: 0.25

🔍 test003_2.jpg
   ✅ Prediksi: B1995 | GT: B1995JVK | CER: 0.375

🔍 test004_1.jpg
   ✅ Prediksi: 93027VEH | GT: B9062VEH | CER: 0.5

🔍 test005_1.jpg
   ✅ Prediksi: 278798KM | GT: DD3798KM | CER: 0.375

🔍 test006_1.jpg
   ✅ Prediksi: 1329KC | GT: T1329KC | CER: 0.143

🔍 test007_1.jpg
   ✅ Prediksi: AD8865EE | GT: AD8865EE | CER: 0.0

🔍 test008_1.jpg
   ✅ Prediksi: DK1167AAB | GT: DK1157AAB | CER: 0.111

🔍 test008_2.jpg
   ✅ Prediksi: AA1997FE | GT: AA1997FE | CER: 0.0

🔍 test009_1.jpg
   ✅ Prediksi: HW8018NA | GT: H8518NA | CER: 0.286

🔍 test009_2.jpg
   ✅ Prediksi: 1649 | GT: K1649GB | CER: 0.429

🔍 test010_1.jpg
   ✅ Prediksi: B9416CPCN | GT: B9416PCN

KeyboardInterrupt: 